# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
from IPython.display import display

m = Mystem()

In [2]:
credit = pd.read_csv('/datasets/data.csv')
credit.info()
display(credit.describe().T)
display(credit.sample(n=10, random_state = 3))
#credit.loc[credit.loc[:, 'days_employed'] < 0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3969,0,356799.793930,59,среднее,1,женат / замужем,0,M,пенсионер,0,282342.307327,на покупку автомобиля
2088,0,336706.356980,59,высшее,0,женат / замужем,0,M,пенсионер,0,115843.211369,сделка с автомобилем
1920,0,376720.094240,57,среднее,1,гражданский брак,1,F,пенсионер,0,88614.805688,сыграть свадьбу
10636,0,-949.003202,27,высшее,0,Не женат / не замужем,4,F,компаньон,0,55584.489585,недвижимость
3267,1,-927.954619,41,высшее,0,женат / замужем,0,M,сотрудник,0,282876.334316,ремонт жилью
5750,0,-768.146103,30,среднее,1,гражданский брак,1,F,сотрудник,0,72030.130951,свадьба
2797,2,-4908.553231,37,высшее,0,гражданский брак,1,M,сотрудник,0,120406.031903,свадьба
7757,0,-4388.014411,31,высшее,0,Не женат / не замужем,4,M,сотрудник,0,197301.422694,свой автомобиль
4554,2,-2896.196190,31,высшее,0,в разводе,3,F,госслужащий,0,95286.472998,образование
11228,0,-1769.534329,47,неоконченное высшее,2,женат / замужем,0,M,компаньон,0,149985.113702,получение высшего образования


**Вывод**

1) Есть пропуски в колонках с ежемесячным доходом и стажем (~10% данных). При этом кол-во строк с заполненными доходом и стажем равно.
2) Стаж и доход записаны в виде чисел с плавающей точкой, в чем нет необходимости. Эти графы можно привести к целочисленному виду для удобства восприятия и сравнения.
3) Значительное число строк имеет отрицательный стаж. Зависит от того, как в анкете вносилась информация о стаже, но возможно из-за того что были перепутаны даты начала и окончания работы, либо же из-за технической ошибки/бага (просто добавился знак "минус"?) - какого-то явного паттерна с отрицательными значениями с первого взгляда не прослеживается. Поскольку по условиям задачи нет необходимости анализировать зависимость от стажа, то можно оставить эти значения в исходном виде.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
#df.loc[df.loc[:, 'days_employed'].isna() == True]
credit=credit.fillna(0)
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Явной закономерности в строках с пустым доходом и стажем не прослеживается. Удалять эти строки скорее всего не целесообразно, поскольку в этом случае мы потеряем часть данных при анализе зависимости возврата кредитов от факторов, не связанных с доходом. Для того, чтобы не создавать проблем с вычислениями, заменим пропуски на нули.

### Замена типа данных

In [4]:
credit['days_employed'] = credit['days_employed'].astype('int32')
credit['total_income'] = credit['total_income'].astype('int32')
credit['children'] = credit['children'].astype('int8')
credit['dob_years'] = credit['dob_years'].astype('int8')
credit['education_id'] = credit['education_id'].astype('int8')
credit['family_status_id'] = credit['family_status_id'].astype('int8')
credit['debt'] = credit['debt'].astype('bool')
credit['gender'] = credit['gender'].astype('category')
display(credit.info())
display(credit.sample(n=10, random_state=3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int8
days_employed       21525 non-null int32
dob_years           21525 non-null int8
education           21525 non-null object
education_id        21525 non-null int8
family_status       21525 non-null object
family_status_id    21525 non-null int8
gender              21525 non-null category
income_type         21525 non-null object
debt                21525 non-null bool
total_income        21525 non-null int32
purpose             21525 non-null object
dtypes: bool(1), category(1), int32(2), int8(4), object(4)
memory usage: 967.2+ KB


None

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3969,0,356799,59,среднее,1,женат / замужем,0,M,пенсионер,False,282342,на покупку автомобиля
2088,0,336706,59,высшее,0,женат / замужем,0,M,пенсионер,False,115843,сделка с автомобилем
1920,0,376720,57,среднее,1,гражданский брак,1,F,пенсионер,False,88614,сыграть свадьбу
10636,0,-949,27,высшее,0,Не женат / не замужем,4,F,компаньон,False,55584,недвижимость
3267,1,-927,41,высшее,0,женат / замужем,0,M,сотрудник,False,282876,ремонт жилью
5750,0,-768,30,среднее,1,гражданский брак,1,F,сотрудник,False,72030,свадьба
2797,2,-4908,37,высшее,0,гражданский брак,1,M,сотрудник,False,120406,свадьба
7757,0,-4388,31,высшее,0,Не женат / не замужем,4,M,сотрудник,False,197301,свой автомобиль
4554,2,-2896,31,высшее,0,в разводе,3,F,госслужащий,False,95286,образование
11228,0,-1769,47,неоконченное высшее,2,женат / замужем,0,M,компаньон,False,149985,получение высшего образования


**Вывод**

Проблемы с типами данных наблюдаются в колонках с доходом и стажем. Для удобства вычислений заменим вещественный тип на целочисленный, для этого хорошо подойдет функция astype, поскольку нам нужно привести уже числовые данные к конкретному типу int. Колонку debt можно привести к типу boolean. Кроме того, для большинства колонок хватит размера int8 - это позволит уменьшить размер таблицы в памяти. 

### Обработка дубликатов

In [5]:
#Проверим, какие значения принимают различные поля
credit['education'].value_counts()
credit['education'] = credit['education'].str.lower()
credit['education'].value_counts()
#credit['family_status'].value_counts()
#credit['income_type'].value_counts()
#credit['children'].value_counts()
#credit.loc[credit.loc[:, 'children'] == -1] 
#credit.loc[credit.loc[:, 'children'] == 20] 
#credit['children'] = credit['children'].replace(-1,1)
#credit['children'].value_counts()
#credit.loc[credit.loc[:, 'total_income'] < 0] #записей с отрицательным доходом не обнаружено

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

- Обнаружено 47 записей с отрицательным количеством детей (-1). Можно заменить, что в большинстве из них семеное положение - женат. 
Предположим, что '-' это технологическая ошибка, возомжно в поле можно ввети отрицательную величину, заменим значения на "1"
- Есть и другая аномалия - у 67ми заемщиков 20 детей, что не похоже на корректные данные, тк это число выбивается из остального ряда. 
Тут сложно судить, как исправить эти данные. Возможно правильное значение 0, возможно 2. 
Тк в таких случаях опять же, семейное положение отлично от "холост", скорее всего правильным будет говорить, что кол-во детей в этом случае > 0
Поскольку по условиям задачи нет необходимост анализировать количество детей, только их наличие, оставим данные как есть.
По этой причине также можно было бы и не менять и -1 на 1, просто считая что дети есть, когда credit['children'] !=0

**Вывод**

1) Поскольку данные даные без уникальных идентификаторов (заемщика, например), в поиске полных дубликатов смысла нет - в этом случае за дубликаты можно принять случайно совпавшие по всем параметрам записи.
2) Обнаружены дубликаты в графе Образование: записи с одинаковым текстом, но в различном регистрах. Все значения были приведены к нижнему регистру соответствующим методом str.lower(). Причина появления дубликатов возможно кроется либо в том, что это было свободное текстовое поле в анкете (хотя обычно такого не бывает, возможно это иная технологическая причина, какие-то ошибки в коде).

### Лемматизация

In [6]:
#Получим список всех уникальных ключевых слов
#for purpose in credit['purpose']:
#    lemmas += m.lemmatize(purpose) #запишем леммы из всех записей в одну переменную
#lemmas_list = list(set(lemmas)) #для получения списка уникаьных лемм используем структуру set
#lemmas_list


for purpose in credit['purpose']:
   lemmas = m.lemmatize(purpose)
   for word in lemmas:
        if 'операция' == word:
            print(purpose)

операции с жильем
операции с коммерческой недвижимостью
операции с коммерческой недвижимостью
операции со своей недвижимостью
операции с недвижимостью
операции с жильем
операции с коммерческой недвижимостью
операции с коммерческой недвижимостью
операции с коммерческой недвижимостью
операции с недвижимостью
операции со своей недвижимостью
операции с жильем
операции с недвижимостью
операции со своей недвижимостью
операции с коммерческой недвижимостью
операции с коммерческой недвижимостью
операции со своей недвижимостью
операции со своей недвижимостью
операции с недвижимостью
операции с жильем
операции с коммерческой недвижимостью
операции с жильем
операции с коммерческой недвижимостью
операции с коммерческой недвижимостью
операции с недвижимостью
операции с коммерческой недвижимостью
операции с коммерческой недвижимостью
операции с недвижимостью
операции с жильем
операции с жильем
операции с коммерческой недвижимостью
операции со своей недвижимостью
операции с жильем
операции с коммерчес

**Вывод**

Для лемматизации целей кредита использована следующая последовательность шагов:
1) Получение списка уникальных лемм по всей колонке purpose. Есть сразу очевидные цели: образование, свадьба, жилье (покупка, строительство), ремонт. Встречаются и не совсем понятные, например 'заниматься', 'дополнительный', 'высокий', 'подержать'. 
['покупка',
 'заниматься',
 'сделка',
 'дополнительный',
 '\n',
 'с',
 'жилой',
 'жилье',
 'приобретение',
 ' ',
 'операция',
 'недвижимость',
 'со',
 'подержанный',
 'образование',
 'автомобиль',
 'подержать',
 'собственный',
 'на',
 'сдача',
 'для',
 'профильный',
 'получение',
 'коммерческий',
 'свой',
 'проведение',
 'свадьба',
 'семья',
 'сыграть',
 'высокий',
 'строительство',
 'ремонт']
2) Прояснение не очевидных лемм, для чего последовательно сделаем выборки целей, в которых они встречаются. Например, оказалось что 'высокий' и 'дополнительный' относится к образованию. Поэтому можно их отбросить, тк будет иметь значение только ключевое слово 'образование'. Интересно, что "операция" могло бы означать и медицинскую операцию, однако выборка целей с этим словом говорит нам, что во всех случаях имеются ввиду операции с недвижимостью.
3) Выделение конечного списка независимых категорий и соответствующих им ключевых слов (лемм): 
 - 'свадьба'
 - недвижимость, можно условно подразделить на:
    - покупку/строительство собственного жилья ('строительство', 'недвижимость', 'жилье', НЕ включая "коммерческий")
    - покупку жилья под сдачу ('сдача')
    - покупку коммерческой недвижимости ('коммерческий')
    - 'ремонт'
   Скорее всего нельзя на 100% судить о типе недвижимости на основании заявленной цели, поскольку очевидно, что не все заемщики явно указывают тип. К примеру, распространенная цель "операции с недвижимостью" может подрузумевать жилую для себя, под сдачу либо коммерческую. Поэтому отнесем любую покупку и строительство к категории "покупка/строительство недвижимости" ('жилье' или 'недвижимость' и отсутствует 'ремонт') и отдельно выделим "ремонт"
 - 'образование'
 - 'автомобиль'

### Категоризация данных

In [7]:
#Создадим словари
education_dic = credit[['education', 'education_id']]
family_status_dic = credit[['family_status', 'family_status_id']]
education_dic = education_dic.drop_duplicates().reset_index(drop=True)
family_status_dic = family_status_dic.drop_duplicates().reset_index(drop=True)
display(education_dic)
display(family_status_dic)


,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [8]:
#Категоризируем по уровню дохода
#Для определения диапазонов оценим кол-во заемщиков в зависимости от уровня
credit[(credit['total_income'] < 50000) & (credit['total_income'] != 0)].count()
credit[(credit['total_income'] < 100000) & (credit['total_income'] >=50000)].count()
credit[(credit['total_income'] < 150000) & (credit['total_income'] >=100000)].count()
credit[(credit['total_income'] < 200000) & (credit['total_income'] >=150000)].count()
credit[(credit['total_income'] < 300000) & (credit['total_income'] >=200000)].count()
credit[(credit['total_income'] > 300000) & (credit['total_income'] != 0)].count()

#Функция для расчета группы дохода
def income_group(income):
    if income < 50000 and income != 0:
        return 'до 50тыс'
    if income < 100000 and income >= 50000:
        return 'от 50 до 100тыс'
    if income < 150000 and income >= 100000:
        return 'от 100 до 150тыс'
    if income < 200000 and income >= 150000:
        return 'от 150 до 200тыс'
    if income < 300000 and income >= 200000:
        return 'от 200 до 300тыс'
    if income > 300000 and income !=0:
        return 'свыше 300тыс'
    else:
        return 'не указан'
    
display(income_group(100000))

'от 100 до 150тыс'

In [9]:
#Функция для рачета наличия детей
def children_group(children):
    if children != 0:
        return 'Дети есть'
    else:
        return 'Детей нет'

display(children_group(1))

'Дети есть'

In [10]:
#Функция для расчета цели
def purpose_group(purpose):
    lemmas = m.lemmatize(purpose)
    if 'свадьба' in lemmas:
        return 'Свадьба'
    if ('жилье' in lemmas or 'недвижимость' in lemmas) and 'ремонт' not in lemmas:
        return 'Покупка/строительство недвижимости'
    if 'ремонт' in lemmas:
        return 'Ремонт'
    if 'образование' in lemmas:
        return 'Образование'
    if 'автомобиль' in lemmas:
        return 'Автомобиль'

#Тестирование
display(purpose_group('операции с жильем'))

'Покупка/строительство недвижимости'

In [11]:
#Заполнение колонок с группами в датафрейме
credit['income_group'] = credit['total_income'].apply(income_group)
credit['children_group'] = credit['children'].apply(children_group)
credit['purpose_group'] = credit['purpose'].apply(purpose_group)
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 15 columns):
children            21525 non-null int8
days_employed       21525 non-null int32
dob_years           21525 non-null int8
education           21525 non-null object
education_id        21525 non-null int8
family_status       21525 non-null object
family_status_id    21525 non-null int8
gender              21525 non-null category
income_type         21525 non-null object
debt                21525 non-null bool
total_income        21525 non-null int32
purpose             21525 non-null object
income_group        21525 non-null object
children_group      21525 non-null object
purpose_group       21525 non-null object
dtypes: bool(1), category(1), int32(2), int8(4), object(7)
memory usage: 1.4+ MB


**Вывод**

С целью нахождения ответов на поставленные вопросы, для построения сводных таблиц записи были категоризированы: по уровню дохода заемщика (<50тыс, 50-100тыс, 100-150тыч, 150-200тыс, 200-300тыс, >300тыс, не указан), по наличию детей и по цели кредита. По уровню дохода было сделано достаточно много категорий, чтобы была возможность оценить результаты с высокой степенью детализации.
Также мы выделили словари по образованию и семейному положению. В целом, в данном случае необходимости в такой нормализации данных нет, тк сводные отчеты удобно строить по словарным значениям из колонок в самой таблице.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [12]:
#Посчитаем среднюю долю должников по всем группам, как справочный показатель
average_debt = credit.loc[credit.loc[:,'debt'] == 1]['debt'].count() / credit['debt'].count()
print('Средняя доля заемщиков-должников по всем группам составляет','{:.1%}'.format(average_debt))
#Посчитаем долю должников в зависимости от наличия детей
credit_pivot = credit.pivot_table(index = 'children_group', columns = 'debt', values = 'total_income', aggfunc = 'count')
credit_pivot['debt_ratio'] = credit_pivot[1]/(credit_pivot[0] + credit_pivot[1])
credit_pivot

Средняя доля заемщиков-должников по всем группам составляет 8.1%


debt,False,True,debt_ratio
children_group,,,
Детей нет,13086,1063,0.075129
Дети есть,6698,678,0.091920


**Вывод**

Среди заемщиков с детьми процент задолжавших составляет 9,2%, тогда как у заемщиков без детей этот показатель составляет 7,5%. Данные показывают, что наличие детей негативно влияет на возврат кредита в срок. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [13]:
credit_pivot = credit.pivot_table(index = 'family_status', columns = 'debt', values = 'total_income', aggfunc = 'count')
credit_pivot['debt_ratio'] = credit_pivot[1]/(credit_pivot[0]+credit_pivot[1])
credit_pivot.sort_values(by='debt_ratio', ascending = True)

debt,False,True,debt_ratio
family_status,,,
вдовец / вдова,897,63,0.065625
в разводе,1110,85,0.071130
женат / замужем,11449,931,0.075202
гражданский брак,3789,388,0.092890
Не женат / не замужем,2539,274,0.097405


**Вывод**

Среди состоящих в браке и тех, которые были в браке ранее, доля добросовестных заемщиков выше, чем среди тех кто не женат или состоит в гражданском браке. Для не женатых процент невозврата в срок состовляет 9,7%, тогда как среди женатых - 7.5%. Примечательно, что наиболее добросовестными по статистике являются заемщики в разводе (7,1%) и овдовевшие (6,6%). У состоящих в гражданском браке показатель невозвратов почти равен неженатым (чуть меньше) - 9,3%.
На основе данных можно сделать вывод, что наличие брака положительно сказывается на возврате кредитов в срок.
Интересно при этом, что наличие детей показало обратный эффект. Чтобы посмотреть на картину чуть более глубоко, посмотрим на разбивку по наличию детей и семейному положению одновременно.

In [14]:
credit_pivot = credit.pivot_table(index = ['children_group', 'family_status'], columns = 'debt', values = 'total_income', aggfunc = 'count')
credit_pivot['debt_ratio'] = credit_pivot[1]/(credit_pivot[0]+credit_pivot[1])
credit_pivot

debt                                  False  True  debt_ratio
children_group family_status                                 
Детей нет      Не женат / не замужем   2055   210    0.092715
               в разводе                729    55    0.070153
               вдовец / вдова           795    53    0.062500
               гражданский брак        2523   229    0.083212
               женат / замужем         6984   516    0.068800
Дети есть      Не женат / не замужем    484    64    0.116788
               в разводе                381    30    0.072993
               вдовец / вдова           102    10    0.089286
               гражданский брак        1266   159    0.111579
               женат / замужем         4465   415    0.085041

Видно, что среди заемщиков с детьми показатель должников высок за счет не женатых: 11.2% у состоящих в гражданском браке и 11,7% у неженатых.
У состоящих в официальном браке и имеющих детей заемщиков процент невозратов составляет 8,5%, что немного превышает среднестатистический показатель (8,1%) и выше, чем у бездетных семей (6,9%).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [15]:
credit_pivot = credit.pivot_table(index = 'income_group', columns = 'debt', values = 'total_income', aggfunc = 'count')
credit_pivot['debt_ratio'] = credit_pivot[1]/(credit_pivot[0]+credit_pivot[1])
credit_pivot.sort_values(by='debt_ratio', ascending = True)

debt,False,True,debt_ratio
income_group,,,
до 50тыс,349,23,0.061828
от 200 до 300тыс,3332,252,0.070312
свыше 300тыс,1376,106,0.071525
не указан,2004,170,0.078197
от 50 до 100тыс,3760,331,0.080909
от 100 до 150тыс,5213,491,0.086080
от 150 до 200тыс,3750,368,0.089364


**Вывод**

Сильной корреляции между уровнем дохода и возвратом в срок не наблюдается. В целом, можно отметить, что наиболее добросовестные заемщики встречаются с дохом ниже среднего (хотя выборка с доходом до 50тыс слишком мала) и высоким (свыше 200тыс). Среди основной же массы заемщиков доля невозвратов примерно одинакова независимо от дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [16]:
credit_pivot = credit.pivot_table(index = 'purpose_group', columns = 'debt', values = 'total_income', aggfunc = 'count')
credit_pivot['debt_ratio'] = credit_pivot[1]/(credit_pivot[0]+credit_pivot[1])
credit_pivot.sort_values(by='debt_ratio', ascending = True)

debt,False,True,debt_ratio
purpose_group,,,
Ремонт,577,35,0.057190
Покупка/строительство недвижимости,9481,747,0.073035
Свадьба,2162,186,0.079216
Образование,3652,370,0.091994
Автомобиль,3912,403,0.093395


**Вывод**

Наиболее рискованно давать кредит под образование и покупку автомобиля: доля должников по таким категориям составляет 9.2% и 9.3% соответственно.
Среди заемов на недвижимость доля долга меньше всего - 7.3% под покупку или строительство и всего %5.7 у тех, кто берет под ремонт (стоит оговориться, что выборка кредитов под ремонт статистически мала).

## Шаг 4. Общий вывод

Данные показали, что на возврат кредита в срок влияют следующие параметры:
- Семейное положение. Наличие официального брака положительно сказывается на возврате кредита в срок. Для не женатых процент невозврата в срок составляет 9,7%, для женатых - 7.5%. Примечательно, что наиболее добросовестными по статистике являлись заемщики в разводе (7,1%) и овдовевшие (6,6%).
- Наличие детей. Среди заемщиков с детьми процент задолжавших составляет в среднем 9,2%, тогда как у тех кто без детей этот показатель составляет 7,5%. Больше всего должников среди имеющих детей и при этом состоящих в гражданском браке (11.2%) и неженатых (11,7%). 
- Цель кредита. Наибольшая доля должников - среди тех, кто берет заем под образование (9.2%) и покупку авто (9.3%). По займам на недвижимость доля долга меньше всего: 7.3% под покупку или строительство и всего %5.7 у тех, кто берет под ремонт

Устойчивого влияния уровня дохода на возврат кредита в срок выявлено не было.